<a href="https://colab.research.google.com/github/ras0k/auto-lyrics/blob/main/Auto_Lyrics_v9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Whisper transcription
import subprocess
import sys

# Install necessary packages quietly to minimize output
print("Installing packages, please wait...")
!pip install yt-dlp > /dev/null 2>&1
!pip install git+https://github.com/openai/whisper.git > /dev/null 2>&1
!pip install ipywidgets > /dev/null 2>&1

# Import necessary modules
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import output
from google.colab import files
import os

# Clear the output to display widgets first
clear_output(wait=True)

# Create widgets
option_dropdown = widgets.Dropdown(
    options=['Download from YouTube URLs', 'Upload audio files'],
    value='Download from YouTube URLs',
    description='Option:',
    layout={'width': 'max-content'}
)

text_area = widgets.Textarea(
    value='',
    placeholder='Enter YouTube URLs separated by new lines',
    description='Input:',
    layout={'width': '100%', 'height': '200px'}
)

process_button = widgets.Button(description="Process")

# Output area for logs
output_area = widgets.Output()

# Function to handle processing
def on_process_clicked(b):
    with output_area:
        clear_output(wait=True)
        def run_command(command):
            """Run a shell command and display output in real-time."""
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
            for line in iter(process.stdout.readline, ''):
                print(line, end='')
                sys.stdout.flush()
            process.wait()

        def transcribe_audio(file_paths):
            for i, file_path in enumerate(file_paths, 1):
                if file_path.strip():
                    print(f"\nTranscribing audio: {file_path}")
                    whisper_command = f"whisper '{file_path}' --model large"
                    run_command(whisper_command)
                    print(f"Transcription completed for {file_path}\n")

        def download_and_transcribe(urls):
            urls = urls.strip().split('\n')
            file_paths = []
            for i, url in enumerate(urls, 1):
                if url.strip():
                    audio_file = f"audio_{i}"
                    print(f"Downloading audio from: {url}")
                    yt_dlp_command = f"yt-dlp -x -o '{audio_file}.%(ext)s' --audio-format mp3 --force-overwrites '{url}'"
                    run_command(yt_dlp_command)
                    downloaded_files = [f for f in os.listdir('.') if f.startswith(audio_file)]
                    file_paths.extend(downloaded_files)
            transcribe_audio(file_paths)

        def upload_and_transcribe():
            print("Please upload your audio files:")
            uploaded = files.upload()
            file_paths = list(uploaded.keys())
            transcribe_audio(file_paths)

        if option_dropdown.value == 'Download from YouTube URLs':
            if text_area.value.strip():
                download_and_transcribe(text_area.value)
            else:
                print("Please enter at least one YouTube URL.")
        else:
            upload_and_transcribe()

process_button.on_click(on_process_clicked)

# Function to update displayed widgets based on selection
def on_option_change(change):
    if change['new'] == 'Download from YouTube URLs':
        text_area.layout.display = 'block'
    else:
        text_area.layout.display = 'none'

option_dropdown.observe(on_option_change, names='value')

# Initial display of widgets
display(option_dropdown, text_area, process_button, output_area)

In [ ]:
# @title Collect all .txt files and zip them (optional)

import os
import zipfile

def zip_txt_files(directory, zip_filename):
  """Zips all .txt files in a directory."""
  with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(directory):
      for file in files:
        if file.endswith(".txt"):
          filepath = os.path.join(root, file)
          zipf.write(filepath, os.path.relpath(filepath, directory))

# Example usage:
zip_txt_files(".", "txt_files.zip")  # Zip all .txt files in the current directory
print("TXT files zipped successfully.")

In [ ]:
# @title Delete all files to reset

import os
import glob

# Remove files starting with "audio" or "txt"
files_to_remove = glob.glob('audio*') + glob.glob('txt*')
for file in files_to_remove:
    try:
        os.remove(file)
        print(f"Removed: {file}")
    except OSError as e:
        print(f"Error removing {file}: {e}")

print("File cleanup complete.")